In [57]:
import pandas as pd
import numpy as np
import ast
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.schema import TextNode

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
# set up OpenAI
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

In [4]:
import chromadb

In [ ]:
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("quickstart")

###Dataset Preparation

In [4]:
df= pd.read_csv('./../data/cleaned/template_bullet.csv')

In [5]:
df.head(5)

,asin_id,title,context,filter,bullet_points
0,B07MZCMZNG,Copco Hydra Reusable Tritan Water Bottle Spill...,"['BABY_BOTTLE', 'material_type_free:BPA Free',...",['binding:miscellaneous'],"bullet_point:[Durable, single-wall, BPA-free T..."
1,B07T175PWC,Nuby Thirsty Kids Push Button Flip-it Soft Spo...,"['BABY_BOTTLE', 'binding:baby_product', 'manuf...","['age_range_description:Kid', 'color:Multi', '...",bullet_point:nan
2,B0009RNXNK,J.L. Childress MaxiCOOL 4 Bottle Breastmilk Co...,"['BABY_BOTTLE', 'binding:baby_product', 'manuf...","['material_type_free:Phthalates Free', 'color:...",bullet_point:[Insulated bottle cooler holds up...
3,B06XXTX2H2,Baby Fanatic NCAA Alabama Crimson Tide Unisex ...,"['BABY_BOTTLE', 'material_type_free:BPA Free']","['age_range_description:Kid', 'color:multicolo...",bullet_point:[Officially Licensed Collegiate P...
4,B00JLVEJ88,Dr. Brown's Pink 8 Ounce Bottles (2 Pack),"['BABY_BOTTLE', 'material_type_free:BPA Free',...",['material:BPA Free'],bullet_point:[Innovative Dr. Brown's Natural F...


In [65]:
nodes= []
for index,row in df.iterrows():
    text = row['bullet_points']
    text = text.split(':')[1]
    text = text.strip("[]")
    context_list, filter_list = ast.literal_eval(row['context']), ast.literal_eval(row['filter'])
    meta = {'product_type': context_list[0]}
    for i in range(1,len(context_list)):
        try:
            key,value = context_list[i].split(":")
        except:
            continue
        meta[key] = value
    for i in filter_list:
        try:
            key,value = i.split(":")
        except:
            continue
        meta[key] = value
    nodes.append(
        TextNode(
            text = (text),
            metadata = meta
        )
    )

In [70]:
print(nodes[0].metadata.items())

dict_items([('product_type', 'BABY_BOTTLE'), ('material_type_free', 'BPA Free'), ('age_range_description', 'Baby'), ('color', 'Orange, Yellow, Gray'), ('manufacturer', 'Lifetime Brands'), ('brand', 'Copco'), ('binding', 'miscellaneous')])
